# Introduction to Synthetic Aperture Radar Using Python and MATLAB

## by Andy Harrison - &copy; Artech House 2022

---

## Example 6.8.2 Phase Correlation
---

**Import modules**

In [1]:
import cv2

import numpy as np

from scipy.signal import cheby1, freqs

from scipy.ndimage import map_coordinates

**Helper function for displaying images**

In [2]:
def imshow(image, windowName, nx, ny):
    cv2.namedWindow(windowName, cv2.WINDOW_NORMAL)    # Create a new named window
    cv2.moveWindow(windowName, 0, 0)                  # Put window @ (0, 0)
    cv2.imshow(windowName, image)                     # Display the image
    cv2.resizeWindow(windowName, nx, ny)              # Resize the window

**Log polar converstion**

In [3]:
def log_polar(image):
    """
    Calculate the log-polar conversion
    """
    # Image dimensions
    nx, ny = image.shape
    

    # Get the angles using meshgrid
    _, theta = np.meshgrid(np.linspace(0, 1, ny), np.linspace(0, -np.pi, nx))
    

    # Calculate the log base
    log_base = 10.0 ** (np.log10(np.sqrt((0.5 * nx) ** 2 + (0.5 * ny) ** 2)) / ny)
    

    # Calculate the radius based on the log base
    radius = log_base ** np.arange(ny) - 1.0
    

    # Calculate the x and y coordinates
    x = radius * np.sin(theta) + nx / 2
    y = radius * np.cos(theta) + ny / 2
    

    # Map the image onto the coordinates
    return log_base, map_coordinates(image, [x, y])

**Perform scale, translation, and rotation of an image**

In [4]:
def scale_translate_rotate(image, scale_x, scale_y, translate_x, translate_y, rotation):
    """
    Scale, rotate, and translate an image
    """
    height, width = image.shape[:2]

    nh = int(np.round(1.2*height))
    
    nw = int(np.round(1.2*width))


    # Scale
    # image = cv2.resize(image, None, fx=scale_x, fy=scale_y, interpolation=cv2.INTER_CUBIC)
    image = cv2.warpAffine(image, np.float32([[scale_x, 0, 0], [0, scale_y, 0]]), (nw, nh))
    

    # Translation
    m = np.float32([[1, 0, translate_x], [0, 1, translate_y]])
    image = cv2.warpAffine(image, m, (nw, nh))
    

    # Rotation
    m = cv2.getRotationMatrix2D(((width - 1) / 2.0, (height - 1) / 2.0), rotation, 1)
    image = cv2.warpAffine(image, m, (nw, nh))
    

    return image

**Read an image**

In [5]:
image1 = cv2.imread('ICEYE_Phase_Correlation.jpg')

**Apply a scale, translation, and rotation**

In [6]:
image2 = scale_translate_rotate(image1, 0.6, 0.6, 200, 100, 30)

**Begin phase correlation - convert to grayscale**

In [7]:
image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)

image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

**Display both images**

In [8]:
imshow(image1, 'Original Image', 600, 600)

imshow(image2, 'Scaled, Translated, Rotated', 600, 600)

**Find the log-polar transformed image and the log base**

In [9]:
f1 = np.fft.fftshift(abs(np.fft.fft2(image1, image2.shape[:2])))

f2 = np.fft.fftshift(abs(np.fft.fft2(image2, image2.shape[:2])))

**2nd order Chebyshev bandstop filter**

In [10]:
[b, a] = cheby1(2, 0.1, [0.01, 0.5], 'bandstop', analog=True)

w, h1 = freqs(b, a, np.linspace(-1, 1, f2.shape[0]))

w, h2 = freqs(b, a, np.linspace(-1, 1, f2.shape[1]))

**Create and apply the 2D filter**

In [11]:
h = np.outer(1 - abs(h1), 1 - abs(h2))

f1 *= (1 - h)

f2 *= (1 - h)

**Find the log-polar transformed image and the log base**

In [12]:
base, log_polar_original = log_polar(f1)

base, log_polar_test = log_polar(f2)

**Calculate the cross-power spectrum of the original and test images**

In [13]:
f1 = np.fft.fft2(log_polar_original)

f2 = np.fft.fft2(log_polar_test)

r0 = abs(f1) * abs(f2)

cross_power_spectrum = abs(np.fft.ifft2((f1 * f2.conjugate()) / r0))

**Calculate the scale and rotation factors**

In [14]:
angle, scale = np.unravel_index(np.argmax(cross_power_spectrum), cross_power_spectrum.shape)


# Calculate the scale factor
scale = base ** scale


# Calculate the rotation angle
angle = 180.0 * angle / cross_power_spectrum.shape[0]


# Adjust the scale factor and rotation angle
if scale > 1.8:
    cross_power_spectrum = abs(np.fft.ifft2((f2 * f1.conjugate()) / r0))
    
    angle, scale = np.unravel_index(np.argmax(cross_power_spectrum), cross_power_spectrum.shape)
    
    
    angle = -180.0 * angle / cross_power_spectrum.shape[0]
    scale = 1.0 / (base ** scale)
    

# Wrap the rotation angle (-90, 90]
angle = (angle + 90) % 180 - 90

**Zoom the test image using the scale factor**

In [15]:
image_transformed = scale_translate_rotate(image2, 1, 1, 0, 0, angle)

image_transformed = scale_translate_rotate(image2, 1./scale, 1./scale, 0, 0, 0)

**Need to handle image shapes for rotated and zoomed images**

In [16]:
nxo, nyo = image1.shape

nxt, nyt = image_transformed.shape

image_transformed = image_transformed[int((nxt - nxo) / 2):int((nxt + nxo) / 2), 
                                      int((nyt - nyo) / 2):int((nyt + nyo) / 2)]

**Calculate the cross-power spectrum of the original and transformed images**

In [17]:
image_original_spectrum = np.fft.fft2(image1)

image_transformed_spectrum = np.fft.fft2(image_transformed)

translation = abs(np.fft.ifft2((image_original_spectrum * image_transformed_spectrum.conjugate()) / 
                               (abs(image_original_spectrum) * abs(image_transformed_spectrum))))

**Find the translation vector**

In [18]:
translate_x, translate_y = np.unravel_index(np.argmax(translation), translation.shape)

**Force translation vector in the range of image indices**

In [19]:
if translate_x > np.floor(nxo / 2):
    translate_x -= nxo

if translate_y > np.floor(nyo / 2):
    translate_y -= nyo

**Translate the transformed imaged**

In [20]:
image_transformed = scale_translate_rotate(image_transformed, 1, 1, translate_x, translate_y, 0)

**Correct the translation vector based on the rotation angle**

In [21]:
if angle > 0.0:
    offset = int(int(image2.shape[1] / scale) * np.sin(np.radians(angle)))
    translate_x, translate_y = translate_y, offset + translate_x

elif angle < 0.0:
    offset = int(int(image2.shape[0] / scale) * np.sin(np.radians(angle)))
    translate_x, translate_y = offset + translate_y, offset + translate_x

**Correct the scale factor**

In [22]:
scale = (image2.shape[1] - 1) / (int(image2.shape[1] / scale) - 1)

**Transform the image and display the results**

In [23]:
image_transformed = scale_translate_rotate(image2, 1./scale, 1./scale, -20, -10, angle)

imshow(image_transformed, 'Transformed Image', 600, 600)

cv2.waitKey()

13